In [2]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

In [6]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

/content/train.json /content/valid.json 1


In [13]:
def preProcess(reviewText):
    # convert string text to lower
    reviewText = reviewText.lower() 
    # replace any number in string text to character 'num'
    reviewText = re.sub(pattern=r"\d", repl=r"num", string=reviewText) 
    # split a string text, use word_tokenize instead of reviewText.split() to handle stopwords
    text_tokens = word_tokenize(reviewText)
    # remove stopwords
    text_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    # stemming
    ps = PorterStemmer()
    reviewStem = [] # list store result after stemming
    for w in text_without_sw:
        reviewStem.append(ps.stem(w))
    return reviewStem

In [14]:
def readFile(myFile):
    with open(myFile) as f: data = json.load(f) # read json file
    # zip review text correspond to overall 
    overall, reviewTex = zip(*[(rev['overall'], preProcess(rev['reviewText'])) for rev in data])
    # example if list1 = ['1', '2', '3'], list2 = ['3', '4', '5']
    # it'll become list3 = ['1', '2', '3', '4', '5', 'unk']
    vocabulary = sorted(set(list(itertools.chain.from_iterable(reviewTex)) + ['unk']))
    return overall, reviewTex, vocabulary

In [18]:
ove1, rev1, vocab1 = readFile(train_set_path) # read train.json file

In [19]:
ove2, rev2, vocab2 = readFile(valid_set_path) # read valid.json file

In [23]:
def word_embed(reviewText, vocab): 
  bow_vector = np.zeros((len(reviewText), len(vocab)), dtype=int) # define bag of words matrix
  # for each review text
  for j in range(len(reviewText)):
    # for each word within the review text
    for w in reviewText[j]:
      # for each word within the vocabulary
      for i, word in enumerate(vocab):
        # if word appear in vocabulary, add 1 to this position
        if word == w: bow_vector[j, i] += 1
  return bow_vector # return bow matrix

In [25]:
convert_overall_int = tuple(int(ele) for ele in ove2) # nomarlize label overall to int

In [26]:
convert_tuple_to_list = list(convert_overall_int) # convert tuple overall to list

In [27]:
convert_list_to_np_array = np.asarray(convert_tuple_to_list) # convert list overall to numpy array

In [28]:
def processOve1(a):
  a[(a >= 1) & (a <= 3)] = -1 # if overall is 1,2,3 -> assign -1
  return a

def processOve2(a):
  a[(a >= 4) & (a <= 5)] = 1 # if overall is 4,5 -> assign 1
  return a

In [29]:
res1 = processOve1(convert_list_to_np_array)
res2 = processOve2(res1) # process label overall

In [32]:
# build linear regression
# our linear regression will be like that: overall = error + coeff * reviewText
new_x = word_embed(rev2, vocab1) # reviewText get from valid.json file, vocab get from train.json file
new_y = np.reshape(res2, (-1, 1)) # reshape array overall to 2d array with 1 column = length of 2d array new_x 

In [33]:
def singleLinearRegression(X, Y):
  nu, de = 0, 0
  for i in range(len(X)):
    # single linear regression: y = ax + b
    # coeff_a = sigma(1->len(X))((xi-mean(x)) / (yi - mean(y))) / sigma(1->len(X))(xi - mean(X))^2
    # coeff_b = mean(y) - coeff of a * mean(x)
    nu += (X[i] - np.mean(X)) * (Y[i] - np.mean(Y))
    de += (X[i] - np.mean(X))**2
  coeff_a = nu / de
  coeff_b = np.mean(Y) - (coeff_a * np.mean(X))
  return coeff_a, coeff_b # return coeff of a and b

In [34]:
coeff_a, coeff_b = singleLinearRegression(new_x, new_y)

In [69]:
# compute root mean square error 
def compute_rmse(X, coeff_b, coeff_a):
  m = len(X)
  RMSE = 0
  for i in range(m):
    y_predict = coeff_b + coeff_a * new_x[i]
    # rsme = sqrt(sigma(1->m)*(1/m)*(y-yi)^2)
    RMSE += (new_y[i] - y_predict)**2
  RMSE = np.sqrt(RMSE/m)
  return RMSE

In [70]:
result_rmse = compute_rmse(new_x, coeff_b, coeff_a)
print("Root mean square error:", np.average(result_rmse))

Root mean square error: 0.6634920592438485


In [71]:
# compute mean absolute error
def compute_mae(X, coeff_b, coeff_a):
  m = len(X)
  MAE = 0
  for i in range(m):
    y_predict = coeff_b + coeff_a * new_x[i]
    # mae = sigma(1->m)((1/m)*abs(yi-y))
    MAE += abs(y_predict - new_y[i])
  MAE /= m
  return MAE

In [72]:
result_mae = compute_mae(new_x, coeff_b, coeff_a) # find accuracy of linear regression model by calculating mean absolute error
random_num = int(random_number) # convert random_number to int
print(rev2[random_num]) # output text preprocess with a random_number from valid.json file
print("M1 -", np.mean(result_mae)) # output accuracy of model M1 

['strong', 'would', 'like', 'peavey', 'mike', "'ll", 'price', '.']
M1 - 0.44023394014546374
